In [1]:
import gluonts
import mxnet

print(gluonts.__version__)
print(mxnet.__version__)

0.4.1
1.4.1


In [2]:
# standard imports
import numpy as np
import pandas as pd

# Vis
%matplotlib inline
import matplotlib.pyplot as plt
from pprint import pprint

# mxnet
import mxnet as mx

# gluonts
from gluonts.dataset.util import to_pandas

# imports from utility function
import utils_gluonts10 as utils
from utils_gluonts10 import get_dataset, dataset_recipes

In [3]:
print(f"Available datasets: {list(dataset_recipes.keys())}")

Available datasets: ['m4_daily', 'm4_daily_domain', 'm4_hourly', 'm4_monthly', 'm4_monthly_atm', 'm4_monthly_dates', 'm4_monthly_demographic', 'm4_monthly_domain', 'm4_monthly_finance', 'm4_monthly_industry', 'm4_monthly_macro', 'm4_monthly_micro', 'm4_monthly_other', 'm4_quarterly', 'm4_quarterly_atm', 'm4_quarterly_dates', 'm4_quarterly_demographic', 'm4_quarterly_domain', 'm4_quarterly_finance', 'm4_quarterly_industry', 'm4_quarterly_macro', 'm4_quarterly_micro', 'm4_quarterly_other', 'm4_weekly', 'm4_weekly_domain', 'm4_weekly_id', 'my_weekly_tm', 'm4_yearly', 'm4_yearly_demographic', 'm4_yearly_domain', 'm4_yearly_finance', 'm4_yearly_industry', 'm4_yearly_macro', 'm4_yearly_micro', 'm4_yearly_other']


In [4]:
dataname = "m4_weekly_id"
dataset = utils.get_dataset(dataname, regenerate=False)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_weekly_id.


In [5]:
# get the first time series in the training set
train_entry = next(iter(dataset.train))
test_entry = next(iter(dataset.test))

# check keys
print(train_entry.keys())
print(test_entry.keys())

dict_keys(['start', 'item_id', 'target', 'feat_static_cat', 'source'])
dict_keys(['start', 'item_id', 'target', 'feat_static_cat', 'source'])


In [9]:
train_entry

{'start': Timestamp('1750-01-04 00:00:00', freq='W-SUN'),
 'item_id': 'W1',
 'target': array([ 1089.2 ,  1078.91,  1079.88, ..., 36565.18, 36565.18, 35397.16],
       dtype=float32),
 'feat_static_cat': array([1]),
 'source': SourceContext(source=Span(path=WindowsPath('C:/Users/TM/.mxnet/gluon-ts/datasets/m4_weekly_id/train/m4_weekly_id_train.json'), line=1), row=1)}

In [11]:
dataset.metadata.feat_static_cat

[<CategoricalFeatureInfo name='feat_static_cat' cardinality='359'>]

In [12]:
[feat_static_cat.cardinality for feat_static_cat in dataset.metadata.feat_static_cat]

['359']

In [14]:
test_series = to_pandas(test_entry)
train_series = to_pandas(train_entry)

fig, ax = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(10, 7))

train_series.plot(ax=ax[0])
ax[0].grid(which="both")
ax[0].legend(["train series"], loc="upper left")

test_series.plot(ax=ax[1])
ax[1].axvline(train_series.index[-1], color='r') # end of train dataset
ax[1].grid(which="both")
ax[1].legend(["test series",c="upper left")
 "end of train series"], lo
plt.show()

SyntaxError: invalid syntax (<ipython-input-14-8a6d39593d1f>, line 13)

In [16]:
# gluonts
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# DeepAREstimator
from gluonts.model.deepar import DeepAREstimator

In [20]:
epochs=1
num_batches = 50


trainer = Trainer(
    ctx=mx.cpu(0),
    epochs=epochs,
    num_batches_per_epoch=num_batches,
    learning_rate=1e-3,
)

estimator = DeepAREstimator(
    trainer=trainer,
    cardinality= [
        feat_static_cat.cardinality 
        for feat_static_cat in dataset.metadata.feat_static_cat
    ],
    context_length=dataset.metadata.prediction_length,
    prediction_length=dataset.metadata.prediction_length,
    freq=dataset.metadata.freq,
    use_feat_static_cat=True
)

predictor = estimator.train(training_data=dataset.train)

forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_samples=100
    )

INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 10.05it/s, avg_epoch_loss=8.05]
INFO:root:Epoch[0] Elapsed time 5.009 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=8.049834
INFO:root:Loading parameters from best epoch (0)
INFO:root:Final loss: 8.049834442138671 (occurred at epoch 0)
INFO:root:End model training


In [21]:
agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
)

Running evaluation: 100%|██████████| 359/359 [00:28<00:00, 12.43it/s]


In [24]:
item_metrics.head()

,item_id,MSE,abs_error,abs_target_sum,abs_target_mean,seasonal_error,MASE,sMAPE,MSIS,QuantileLoss[0.1],...,QuantileLoss[0.5],Coverage[0.5],QuantileLoss[0.6],Coverage[0.6],QuantileLoss[0.7],Coverage[0.7],QuantileLoss[0.8],Coverage[0.8],QuantileLoss[0.9],Coverage[0.9]
0,W1,4.538236e+06,24788.652344,464329.437500,35717.649038,63.683045,29.942340,0.052180,137.420366,6887.323438,...,24788.652344,1.000000,26460.190625,1.000000,26350.694531,1.000000,22363.709375,1.000000,15866.180469,1.000000
1,W2,1.503622e+05,4802.731445,46501.820312,3577.063101,12.059746,30.634218,0.098446,147.796364,729.338525,...,4802.731934,1.000000,4488.651172,1.000000,3962.851904,1.000000,3153.945020,1.000000,1952.472949,1.000000
2,W3,3.805391e+04,1433.019531,128300.796875,9869.292067,33.355166,3.304804,0.011207,59.711390,2569.758008,...,1433.019531,0.384615,1834.119922,0.923077,2719.716211,1.000000,3036.350000,1.000000,2608.618945,1.000000
3,W4,7.526712e+05,8785.753906,38126.421875,2932.801683,353.363468,1.912555,0.241198,53.789792,2385.204175,...,8785.753662,0.076923,10030.333789,0.076923,10944.186035,0.076923,11307.123340,0.230769,10949.039746,0.307692
4,W5,1.684677e+05,4366.634766,23475.000000,1805.769231,705.777154,0.475922,0.191872,10.996876,1186.253516,...,4366.635010,0.230769,4849.282617,0.230769,5131.259888,0.230769,5026.207471,0.307692,4480.213965,0.307692
